In [28]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.nn.functional as f

In [29]:
ds=pd.read_csv('train.csv')

In [30]:
ds=ds.dropna()

In [31]:
ds.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [32]:
features=["Pclass","Age","SibSp","Parch","Fare","Sex","Cabin"]
label=["Survived"]

In [33]:

ds.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [34]:
cabinlen=len(ds["Cabin"].unique())

In [35]:
le=preprocessing.LabelEncoder()

In [36]:
encoded_cabin=le.fit_transform(ds["Cabin"].unique())
unencoded_cabin=ds["Cabin"].unique()

In [37]:
for i in range(cabinlen):
    ds.loc[ds["Cabin"]==unencoded_cabin[i],"Cabin"]=encoded_cabin[i]

In [38]:
# female=0 male=1
ds.loc[ds["Sex"]=="female","Sex"]=0
ds.loc[ds["Sex"]=="male","Sex"]=1

In [39]:
ds.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,72,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,48,S
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,117,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",0,4.0,1,1,PP 9549,16.7000,131,S
11,12,1,1,"Bonnell, Miss. Elizabeth",0,58.0,0,0,113783,26.5500,43,S


In [40]:
X=ds[features]
y=ds[label]


In [41]:
X["SibSp"]=X["SibSp"].astype("float32")
X["Pclass"]=X["Pclass"].astype("float32")
X["Parch"]=X["Parch"].astype("float32")
X["Sex"]=X["Sex"].astype("float32")
X["Cabin"]=X["Cabin"].astype("float32")

C:\Users\Odin Revolution\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Odin Revolution\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Odin Revolution\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [42]:
X.dtypes

Pclass    float32
Age       float64
SibSp     float32
Parch     float32
Fare      float64
Sex       float32
Cabin     float32
dtype: object

In [43]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [44]:
X_train=np.array(X_train)
y_train=np.array(y_train)


In [45]:
X_train=torch.from_numpy(X_train).float()
y_train=torch.from_numpy(y_train).float()

In [46]:
y_train=y_train.type(torch.LongTensor)

In [47]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [48]:
ts=TensorDataset(X_train,y_train)
dl=DataLoader(ts,batch_size=40)

In [49]:
class Mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(7,2)
    def forward(self,xb):
        xb=xb.reshape(-1,7)
        out=self.linear(xb)
        return out

In [50]:
model=Mymodel()

In [51]:
optim=torch.optim.SGD(model.parameters(),lr=0.01)
loss=nn.functional.cross_entropy

In [52]:
l.shape



NameError: name 'l' is not defined

In [129]:
for i in range(535):
    for data,label in dl:
        prde=model(data)
        l=label.squeeze()
        ls=loss(prde,l)
        ls.backward()
        optim.step()
        optim.zero_grad()
    print(ls)


tensor(2.6432, grad_fn=<NllLossBackward>)
tensor(2.0499, grad_fn=<NllLossBackward>)
tensor(4.9909, grad_fn=<NllLossBackward>)
tensor(19.9922, grad_fn=<NllLossBackward>)
tensor(31.4644, grad_fn=<NllLossBackward>)
tensor(12.7671, grad_fn=<NllLossBackward>)
tensor(7.9847, grad_fn=<NllLossBackward>)
tensor(7.5442, grad_fn=<NllLossBackward>)
tensor(10.8697, grad_fn=<NllLossBackward>)
tensor(11.0278, grad_fn=<NllLossBackward>)
tensor(8.2889, grad_fn=<NllLossBackward>)
tensor(12.1606, grad_fn=<NllLossBackward>)
tensor(6.8710, grad_fn=<NllLossBackward>)
tensor(11.7736, grad_fn=<NllLossBackward>)
tensor(7.1160, grad_fn=<NllLossBackward>)
tensor(14.9861, grad_fn=<NllLossBackward>)
tensor(4.1557, grad_fn=<NllLossBackward>)
tensor(8.7016, grad_fn=<NllLossBackward>)
tensor(10.8437, grad_fn=<NllLossBackward>)
tensor(16.1908, grad_fn=<NllLossBackward>)
tensor(3.2581, grad_fn=<NllLossBackward>)
tensor(6.7150, grad_fn=<NllLossBackward>)
tensor(13.7205, grad_fn=<NllLossBackward>)
tensor(23.3416, grad_fn

tensor(3.6202, grad_fn=<NllLossBackward>)
tensor(9.2280, grad_fn=<NllLossBackward>)
tensor(10.3241, grad_fn=<NllLossBackward>)
tensor(17.6398, grad_fn=<NllLossBackward>)
tensor(2.5817, grad_fn=<NllLossBackward>)
tensor(2.1300, grad_fn=<NllLossBackward>)
tensor(5.1359, grad_fn=<NllLossBackward>)
tensor(16.7900, grad_fn=<NllLossBackward>)
tensor(2.0474, grad_fn=<NllLossBackward>)
tensor(8.1353, grad_fn=<NllLossBackward>)
tensor(14.3847, grad_fn=<NllLossBackward>)
tensor(31.3573, grad_fn=<NllLossBackward>)
tensor(12.6600, grad_fn=<NllLossBackward>)
tensor(7.5143, grad_fn=<NllLossBackward>)
tensor(7.6625, grad_fn=<NllLossBackward>)
tensor(12.0983, grad_fn=<NllLossBackward>)
tensor(11.5799, grad_fn=<NllLossBackward>)
tensor(7.6307, grad_fn=<NllLossBackward>)
tensor(11.4891, grad_fn=<NllLossBackward>)
tensor(7.4848, grad_fn=<NllLossBackward>)
tensor(14.1636, grad_fn=<NllLossBackward>)
tensor(4.8375, grad_fn=<NllLossBackward>)
tensor(9.3918, grad_fn=<NllLossBackward>)
tensor(9.9889, grad_fn=<

tensor(7.1892, grad_fn=<NllLossBackward>)
tensor(10.3528, grad_fn=<NllLossBackward>)
tensor(8.7794, grad_fn=<NllLossBackward>)
tensor(14.1255, grad_fn=<NllLossBackward>)
tensor(4.8270, grad_fn=<NllLossBackward>)
tensor(9.8949, grad_fn=<NllLossBackward>)
tensor(9.3605, grad_fn=<NllLossBackward>)
tensor(15.3596, grad_fn=<NllLossBackward>)
tensor(3.5706, grad_fn=<NllLossBackward>)
tensor(9.6251, grad_fn=<NllLossBackward>)
tensor(9.8218, grad_fn=<NllLossBackward>)
tensor(17.3852, grad_fn=<NllLossBackward>)
tensor(2.4765, grad_fn=<NllLossBackward>)
tensor(2.7070, grad_fn=<NllLossBackward>)
tensor(16.1445, grad_fn=<NllLossBackward>)
tensor(25.6717, grad_fn=<NllLossBackward>)
tensor(7.0057, grad_fn=<NllLossBackward>)
tensor(11.0860, grad_fn=<NllLossBackward>)
tensor(11.7210, grad_fn=<NllLossBackward>)
tensor(7.4874, grad_fn=<NllLossBackward>)
tensor(10.8954, grad_fn=<NllLossBackward>)
tensor(8.1476, grad_fn=<NllLossBackward>)
tensor(14.3405, grad_fn=<NllLossBackward>)
tensor(4.7218, grad_fn=<

In [130]:
for data,label in dl:
    d=data
    l=label
    break

In [131]:
label.shape

torch.Size([46, 1])

In [132]:
l.squeeze().shape

torch.Size([46])

In [133]:
i,j=torch.max(model(d),dim=1)

In [134]:
j

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [135]:
len(j)

46

In [136]:
len(l)

46

In [137]:
torch.sum(l==j)/len(j)

tensor(33)

In [138]:
X_test=np.array(X_test)
y_test=np.array(y_test)

In [139]:
X_test=torch.from_numpy(X_test).float()
y_test=torch.from_numpy(y_test).float()

In [140]:
y_test=y_test.type(torch.LongTensor)

In [141]:
tl=TensorDataset(X_test,y_test)
dl=DataLoader(tl,batch_size=75)

In [151]:
optim=torch.optim.SGD(model.parameters(),lr=0.010456701)
loss=nn.functional.cross_entropy
for x in range(500):
    for i,j in dl:
        with torch.no_grad():
            pred=model(i)
            jj=j.squeeze()
            ls=loss(pred,jj)
            _,pre=torch.max(pred,dim=1)
            acc=torch.sum(pre==j).item()/len(j)

print(f"loss={ls} acc={acc}")

loss=9.580245018005371 acc=33.0
